In [ ]:
import numpy as np
import tensorflow
from sklearn.metrics import accuracy_score
#from tensorflow import set_random_seed
from keras.models import load_model
from keras.layers import Input, Embedding, LSTM, Dense,Concatenate,Lambda
from keras.models import Model
import keras
from keras import backend as K
from keras.layers import Dense, Activation, Flatten,TimeDistributed,Dropout,Bidirectional
from keras.layers import RepeatVector,merge,Permute,GlobalMaxPool1D
#from attention_utils import get_activations, get_data_recurrent
from keras.callbacks import EarlyStopping
import _pickle as cPickle
np.random.seed(123)
tensorflow.random.set_seed(2)
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import requests

In [ ]:
df_newfile=pd.read_csv("/content/new_file (2).csv")
df_newfile.head()

,Country,Slow,Moderate,Fast
0,Aruba,0,2,1
1,Afghanistan,0,2,1
2,Angola,0,2,1
3,Albania,0,1,2
4,Andorra,1,0,2


In [ ]:
# create an Empty DataFrame
df_train = pd.DataFrame(columns = ['SN', 'Date', 'Confirmed Cases','Daily Cases','Country']) 

for ind in df_newfile.index:
  countryname=df_newfile['Country'][ind]
  slowphase=df_newfile['Fast'][ind]
  #df_slow stores SN, Date, ConfirmedCases
  dfslow=pd.read_csv('/content/'+countryname+'.csv_split'+str(slowphase)+'.csv')
  #we add a column with country name and another one with daily cases
  dfslow['CountryName']=countryname
  dfslow['Daily Cases'] = dfslow['Confirmed Cases'].diff().fillna(0)
  
  if (countryname=='Japan' or countryname=='India' or countryname=='Indonesia' or countryname=='United States' or countryname=='Brazil'):
    dfslow=dfslow[0:len(dfslow.index)-17]
  
  #now we add this to Train data set
  for ind2 in dfslow.index:
    df_train = df_train.append({'SN' :dfslow['SN'][ind2],'Date':dfslow['Date'][ind2], 'Confirmed Cases' :dfslow['Confirmed Cases'][ind2],'Daily Cases':dfslow['Daily Cases'][ind2],'Country':countryname}, ignore_index = True)
  #df_train.append(dfslow, ignore_index=True)
  #del dfslow


print(df_train)



          SN      Date  Confirmed Cases  Daily Cases       Country
0        216  20200804            132.0          0.0         Aruba
1        217  20200805            171.0         39.0         Aruba
2        218  20200806            263.0         92.0         Aruba
3        219  20200807            396.0        133.0         Aruba
4        220  20200808            509.0        113.0         Aruba
...      ...       ...              ...          ...           ...
10717  90606  20200804         521318.0       4456.0  South Africa
10718  90607  20200805         529877.0       8559.0  South Africa
10719  90608  20200806         538184.0       8307.0  South Africa
10720  90609  20200807         545476.0       7292.0  South Africa
10721  90610  20200808         553188.0       7712.0  South Africa

[10722 rows x 5 columns]


In [ ]:
df_train.to_csv('df_train.csv')

In [ ]:
X_train=[]
Y_train=[]

#now we divide df_train to X_train and Y_train

#for every country in df_newfile
for ind in df_newfile.index:
  countryname=df_newfile['Country'][ind]
  
  for ind2 in dfslow.index:
    df_g=df_train[df_train['Country']==countryname]
    df_g = df_g[df_g['Daily Cases']>0]
    df_g=df_g['Daily Cases']
    data=df_g.values
    for i in range(0,len(data)):
        if i+17<=len(data):
            X_train.append(data[i:i+10])
            Y_train.append(data[i+10:i+17])
    
X_train=np.array(X_train)
Y_train=np.array(Y_train)
print(X_train)
print(Y_train)
print(len(X_train))
print(len(Y_train))

[[   39.    92.   133. ...    81.    96.    79.]
 [   92.   133.   113. ...    96.    79.    75.]
 [  133.   113.    54. ...    79.    75.    54.]
 ...
 [ 8170. 13150. 13104. ...  7232. 11362. 11046.]
 [13150. 13104. 13944. ... 11362. 11046. 11014.]
 [13104. 13944. 12204. ... 11046. 11014. 10107.]]
[[   75.    54.    19. ...    91.    91.    77.]
 [   54.    19.    84. ...    91.    77.    70.]
 [   19.    84.    91. ...    77.    70.    34.]
 ...
 [11014. 10107.  8195. ...  4456.  8559.  8307.]
 [10107.  8195.  5377. ...  8559.  8307.  7292.]
 [ 8195.  5377.  4456. ...  8307.  7292.  7712.]]
322740
322740


In [ ]:
#scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
data=df_train['Confirmed Cases'].values
#print(data.shape)
data=data.reshape((len(data),1))
scaler.fit(data)
X_train= scaler.transform(X_train)
Y_train=scaler.transform(Y_train)

X_train=X_train.reshape((len(X_train),10,1))
print(X_train.shape,Y_train.shape)


(322740, 10, 1) (322740, 7)


In [ ]:
# LSTM architecture
def model_arch():
    lstm_hidden = 32
    initi = keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=123)
    main_input = Input(shape=(10,1), name='main_input')
    lstm_out = LSTM(lstm_hidden,kernel_initializer=initi, recurrent_initializer= initi, bias_initializer=initi)(main_input)
    lstm_out = Dropout(0.45)(lstm_out)
    main_output = (Dense(7, name='main_output')(lstm_out))
#     Adam=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)

    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model

model = model_arch()
earlystop=EarlyStopping(monitor='val_acc', min_delta=0, patience=4, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
callbacks_list = [earlystop]
print("model is building")
bs = 128
model.fit(batch_size=bs,epochs=10,x=X_train,y=Y_train, verbose=1,validation_split=0.1)
print("model building done")
    

model is building
Epoch 1/10
2270/2270 [==============================] - 20s 8ms/step - loss: 5.3720e-06 - val_loss: 8.2467e-07
Epoch 2/10
2270/2270 [==============================] - 17s 7ms/step - loss: 1.5245e-07 - val_loss: 6.1333e-07
Epoch 3/10
2270/2270 [==============================] - 17s 8ms/step - loss: 1.3744e-07 - val_loss: 6.0171e-07
Epoch 4/10
2270/2270 [==============================] - 17s 7ms/step - loss: 1.2948e-07 - val_loss: 7.0163e-07
Epoch 5/10
2270/2270 [==============================] - 17s 7ms/step - loss: 1.1861e-07 - val_loss: 5.7890e-07
Epoch 6/10
2270/2270 [==============================] - 17s 8ms/step - loss: 1.0965e-07 - val_loss: 6.9251e-07
Epoch 7/10
2270/2270 [==============================] - 17s 7ms/step - loss: 1.0665e-07 - val_loss: 6.6870e-07
Epoch 8/10
2270/2270 [==============================] - 15s 7ms/step - loss: 1.0273e-07 - val_loss: 4.2668e-07
Epoch 9/10
2270/2270 [==============================] - 16s 7ms/step - loss: 1.0142e-07 - val_

In [ ]:
# Prepare test data
X_test=[]
Y_test=[]
for ind in df_newfile.index:
  countryname=df_newfile['Country'][ind]
  slowphase=df_newfile['Fast'][ind]
  #df_slow stores SN, Date, ConfirmedCases
  df2=pd.read_csv('/content/'+countryname+'.csv_split'+str(slowphase)+'.csv')
  #we add a column with country name and another one with daily cases
  df2['CountryName']=countryname
  df2['Daily Cases'] = df2['Confirmed Cases'].diff().fillna(0)
  
  if (countryname=='Japan' or countryname=='India' or countryname=='Indonesia' or countryname=='United States' or countryname=='Brazil'):
    df2=df2[len(df2.index)-1-17:len(df2.index)]
    df2 = df2[df2['Daily Cases']>0]
    df3=df2['Daily Cases']
    data2=df3.values
    #print(len(data2),"\n")
    for i in range(0,len(data2)):
        if i+17<=len(data2):
            X_test.append(data2[i:i+10])
            Y_test.append(data2[i+10:i+17])


X_test=np.array(X_test)
Y_test=np.array(Y_test)
print(len(X_test),len(Y_test))

10 10


In [ ]:
X_test= scaler.transform(X_test)
X_test=X_test.reshape((len(X_test),10,1))
print(X_test.shape,Y_test.shape)

(10, 10, 1) (10, 7)


In [ ]:
# prediction
pred=model.predict(x=X_test)
print(pred.shape)
#inverse scaling
pred1=scaler.inverse_transform(pred)

#Find RMSE
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(Y_test, pred1)
print(np.sqrt(rms))

(10, 7)
23621.53510056644
